# Machine Learning

In [1]:
# global imports
import pyspark
from pyspark.sql.session import SparkSession
# evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.getOrCreate()

# Import Data

In [2]:
# import data
%store -r dfs

In [3]:
train = spark.createDataFrame(dfs[0])
train.toPandas().head()

,count_vec,label_idx
0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...",1.0
2,"(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",0.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",1.0
4,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0


In [4]:
test = spark.createDataFrame(dfs[1])
test.toPandas().head()

,count_vec,label_idx
0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
1,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
2,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
3,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",0.0
4,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0


#### NOTE:

label_idx = 0 is a negative tweet, and label_idx = 1 is a positive tweet. 

# Build Model

## Notes

- I did some research, and since this is an imbalanced classification problem, it's best to use `areaUnderPR` as the metric for evaluating model performance
    - When doing cross validation and assessing the model at the end, please use the variable `class_eval` as the evaluator since it is set up to evaluate based on `areaUnderPR`
- Use 5 fold CV so all the models are trained consistently, and `setSeed()` with a random seed to get reproducible results

I hope these notes help make building the models easier and smoother :) 

## Models to Build
- Logistic Regression
    - Lasso, Ridge, Elastic Net, and Logistic Regression (no penalization)
- Decision Tree
- Random Forest
- Gradient Boosted Tree
- Multilayer Perceptron
- Linear Support Vector Machine
- Naive Bayes

Note that not all of these models are likely to work with our data. If you get weird results (like classifying every observation as `0`), we can choose not to talk about it in the final project, so don't spend too much time worrying if a model does particularly badly :)

- For example, I messed around with Logistic Regerssion a little (didn't hyperparameter tune though) and it seems like it's not going to work with our data
- I did look at Gradient Boosted Tree, and without hyperparameter tuning, it classifies correctly 75% of the time, so there definitely are models that will work so no need to worry about not getting any results :) 

In [5]:
# classification evaluator
class_eval =  BinaryClassificationEvaluator().setLabelCol('label_idx').setMetricName('areaUnderPR')

# Testing different models

## 1) Random Forest

In [6]:
# machine learning transformations
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
# machine learning models
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
# evaluators
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
# cross validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# pipeline
from pyspark.ml import Pipeline

In [10]:
# create random forest model
rf_cv = RandomForestClassifier(featuresCol='count_vec', labelCol='label_idx', featureSubsetStrategy='sqrt')\
                              .setSeed(135)
# parameter grid
pars21 = ParamGridBuilder().addGrid(rf_cv.numTrees,[10,100,500,1000]).build()

# classification evaluator (done at beginning)

# create cross validator
cv_rf = CrossValidator().setEstimator(rf_cv)\
                        .setEvaluator(class_eval)\
                        .setEstimatorParamMaps(pars21)\
                        .setNumFolds(5)\
                        .setSeed(135)

In [11]:
# CODE FOR RUNNING THE CROSS VALIDATION
# run cross validation
cv_rf_model = cv_rf.fit(train)
# get best model
best_rf = cv_rf_model.bestModel
# output best parameter
best_rf._java_obj.getNumTrees()

500

In [12]:
# RUNNING CROSS VALIDATION A SECOND TIME (TO CHECK)

# create random forest model (again)
rf_cv = RandomForestClassifier(featuresCol='count_vec', labelCol='label_idx', featureSubsetStrategy='sqrt')\
                              .setSeed(135)
# parameter grid (again)
pars22 = ParamGridBuilder().addGrid(rf_cv.numTrees,[400,500,550,600]).build()

# classification evaluator (done at beginning)

# create cross validator (again)
cv_rf = CrossValidator().setEstimator(rf_cv)\
                        .setEvaluator(class_eval)\
                        .setEstimatorParamMaps(pars22)\
                        .setNumFolds(5)\
                        .setSeed(135)

In [13]:
# run cross validation
cv_rf_model = cv_rf.fit(train)
# get best model
best_rf = cv_rf_model.bestModel
# output best parameter
best_rf._java_obj.getNumTrees()

500

In [ ]:
# TRAINING THE MODEL

In [14]:
# create the final model
rf = RandomForestClassifier(featuresCol='count_vec', labelCol='label_idx', featureSubsetStrategy='sqrt', numTrees=500).setSeed(135)
# train the model
rf_model = rf.fit(train)
# obtain predictions for test
test_rf = rf_model.transform(test)

In [16]:
# create confusion matrix
confusion_matrix_rf = test_rf.select('label_idx','prediction')
# ‘label’ is row and ‘prediction’ is column
confusion_matrix_rf.stat.crosstab('label_idx','prediction').toPandas()

,label_idx_prediction,0.0,1.0
0,1.0,1364,229
1,0.0,2751,28


Out of 4,372 observations, it misclassified 1,392. This means it had 68% accuracy.


## 2) Gradient Boosted Tree

In [35]:
# create gbt model
gbt_cv = GBTClassifier(featuresCol='count_vec', labelCol='label_idx').setSeed(135)
# parameter grid
pars23 = ParamGridBuilder().addGrid(gbt_cv.stepSize,[0.1,0.5,0.9])\
                           .addGrid(gbt_cv.maxIter,[50,100,200])\
                           .build()
# create cross validator
cv_gbt = CrossValidator().setEstimator(gbt_cv)\
                         .setEvaluator(class_eval)\
                         .setEstimatorParamMaps(pars23)\
                         .setNumFolds(5)\
                         .setSeed(135)

NameError: name 'class_eval' is not defined

In [21]:
# CROSS VALIDATION (TAKES A LONG TIME TO RUN!)

# run cross validation
cv_gbt_model = cv_gbt.fit(train)
# get best model
best_gbt = cv_gbt_model.bestModel
# output best parameters
print('Best Step Size: ',best_gbt._java_obj.getStepSize())
print('Best Number of Trees: ',best_gbt._java_obj.getMaxIter())

Best Step Size:  0.1
Best Number of Trees:  200


In [23]:
# create the final model
gbt = GBTClassifier(featuresCol='count_vec', labelCol='label_idx', stepSize=0.1,maxIter=200).setSeed(135)
# train the model
gbt_model = gbt.fit(train)
# obtain predictions for test
test_gbt = gbt_model.transform(test)

In [24]:
# create confusion matrix
confusion_matrix_gbt = test_gbt.select('label_idx','prediction')
# ‘label’ is row and ‘prediction’ is column
confusion_matrix_gbt.stat.crosstab('label_idx','prediction').toPandas()

,label_idx_prediction,0.0,1.0
0,1.0,591,1002
1,0.0,2413,366


Out of 4,372 observations it misclassified 957. This means it had 78% accuracy.

## 3) Naive Bayes

In [7]:
from pyspark.ml.classification import NaiveBayes

In [14]:
# train a Naive Bayes model
nb = NaiveBayes(labelCol="label_idx", featuresCol="count_vec")

# parameter grid
parsnb = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()

# create cross validator
cv_nb = CrossValidator().setEstimator(nb)\
                         .setEvaluator(class_eval)\
                         .setEstimatorParamMaps(parsnb)\
                         .setNumFolds(5)\
                         .setSeed(135)

In [17]:
# run cross validation
cv_nb_model = cv_nb.fit(train)
# get best model
best_nb = cv_nb_model.bestModel

In [20]:
test_nb = best_nb.transform(test)

In [21]:
# create confusion matrix
confusion_matrix_nb = test_nb.select('label_idx','prediction')
# ‘label’ is row and ‘prediction’ is column
confusion_matrix_nb.stat.crosstab('label_idx','prediction').toPandas()

,label_idx_prediction,0.0,1.0
0,1.0,522,1071
1,0.0,2328,451


Out of 4,372 observations it misclassified 973. This means it had 77.74% accuracy which is very similar to GBT.

## 4) Multilayer Perceptron

In [22]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [46]:
# multilayer perceptron model
mp = MultilayerPerceptronClassifier(maxIter=100, blockSize=128, featuresCol='count_vec', labelCol='label_idx', layers=[4, 5, 4, 2]).setSeed(135)

# parameter grid
#parsmp = ParamGridBuilder().addGrid(mp.layers, [[4, 5, 4, 2],[4, 5, 5, 2],[5, 4, 4, 2]]).build()
    
# create cross validator
#cv_mp = CrossValidator().setEstimator(mp)\
#                        .setEvaluator(class_eval)\
#                        .setNumFolds(5)\
#                        .setSeed(135)

In [47]:
# run cross validation
cv_mp_model = mp.fit(train)

In [48]:
test_mp = cv_mp_model.transform(test)

# create confusion matrix
confusion_matrix_mp = test_mp.select('label_idx','prediction')
# ‘label’ is row and ‘prediction’ is column
confusion_matrix_mp.stat.crosstab('label_idx','prediction').toPandas()

Py4JJavaError: An error occurred while calling o17809.crosstab.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4384.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4384.0 (TID 4239, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon